## <span style = "color:brown"><span style = "font-family: Times New Roman"><p style="text-align: center"> Training and testing your Model</span>

In [19]:
import nbimporter
import os
import glob
import pandas as pd
import Event_Planner_RAG
from langchain.schema import Document
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [20]:
# Define database directory
DB_NAME = "vector_db"

# Initialize or Load Vectorstore
if not os.path.exists(DB_NAME):
    print("⚠️ Vectorstore does not exist. Creating a new one...")
    vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)
else:
    vectorstore = Chroma(persist_directory=DB_NAME, embedding_function=embeddings)
    print("Loaded existing vectorstore.")

Loaded existing vectorstore.


In [21]:
training_data = [
    {"question": "How do I plan a wedding on a budget?", "answer": "Prioritize key expenses, negotiate with vendors, and consider off-season discounts."},
    {"question": "What are some creative corporate event ideas?", "answer": "Try themed networking events, team-building retreats, or virtual hybrid experiences."},
    {"question": "How do I choose the right catering service?", "answer": "Consider dietary restrictions, sample menus, and read past customer reviews."},
    {"question": "What factors should I consider when selecting a venue?", "answer": "Capacity, location, parking, amenities, and ambiance play key roles."},
    {"question": "How can I make my event more sustainable?", "answer": "Use eco-friendly decor, digital invitations, and partner with green vendors."},
    {"question": "What is today's date?", "answer": "21st February 2025"},
    {"question": "when was the last wedding?", "answer": "The last wedding is scheduled for 2025-02-25, which is for Amelia Young and Jack Adams."}
]

In [22]:
# Convert Q&A Pairs into LangChain Documents

documents = [
    Document(page_content=item["question"] + " " + item["answer"], metadata={"source": "training"})
    for item in training_data
]

# Generate Embeddings & Store in Vector Database
print("Adding new training data to vectorstore...")
vectorstore.add_documents(documents)
print(f"Training Complete! {len(training_data)} new Q&A pairs added.")

# Save Updates to ChromaDB
vectorstore.persist_directory = DB_NAME 

print("Vectorstore saved successfully.")

Adding new training data to vectorstore...
Training Complete! 7 new Q&A pairs added.
Vectorstore saved successfully.


In [23]:
if vectorstore._collection.count() == 0:
    print("⚠️ No documents found in the collection. Adding training data...")
    vectorstore = Chroma.from_documents(documents, embedding=embeddings, persist_directory=DB_NAME)
else:
    print(f"✅ Vectorstore contains {vectorstore._collection.count()} documents.")

✅ Vectorstore contains 36 documents.


In [24]:
def test_model():
    #from rag_app import conversation_chain  # Import the chatbot pipeline

    test_questions = [
        "How do I pick a venue for my event?",
        "What are some low-cost wedding planning tips?",
        "How do I ensure my event is environmentally friendly?"
    ]

    for question in test_questions:
        response = conversation_chain.invoke({"question": question})
        print(f"\n🤖 Q: {question}\n💡 A: {response['answer']}")

if __name__ == "__main__":
    test_model()
    launch_chatbot()



🤖 Q: How do I pick a venue for my event?
💡 A: When selecting a venue for your event, consider the following factors: capacity, location, parking, amenities, and ambiance.

🤖 Q: What are some low-cost wedding planning tips?
💡 A: Prioritize key expenses, negotiate with vendors, and consider off-season discounts.

🤖 Q: How do I ensure my event is environmentally friendly?
💡 A: Use eco-friendly decor, digital invitations, and partner with green vendors.
Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.
